In [30]:
import numpy as np
import pandas as pd
import pandas_ta as ta

import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [12, 8]
plt.style.use('fivethirtyeight')

# from sklearn.preprocessing import MinMaxScaler
# from sklearn.pipeline import Pipeline
# from sklearn.model_selection import train_test_split , GridSearchCV , cross_val_score , TimeSeriesSplit , RandomizedSearchCV

# from sklearn.ensemble import RandomForestClassifier

# from sklearn.metrics import classification_report , confusion_matrix , accuracy_score
# from sklearn.metrics import roc_curve , plot_roc_curve , plot_confusion_matrix , auc


import cufflinks as cf
cf.set_config_file(offline=True)
#https://github.com/santosjorge/cufflinks

import seaborn as sns
import plotly.graph_objs as go


import yfinance as yf

In [31]:
tech = ['DELTA', 'INTUCH']
energy = ['BCP', 'BGRIM', 'EA', 'GPSC', 'GULF', 'PTT', 'PTTEP', 'PTTGC', 'TOP','OR']
healthCare = ['BDMS', 'BH']
PublicUtility = ['BEM', 'BJC', 'BTS', 'CENTEL', 'CPALL', 'CPF', 'ERCO', 'HMPRO', 'RATCH','CRC']
Finance = ['BBL', 'CPN', 'KBANK', 'KTB', 'KTC', 'LH', 'MTC', 'TISCO', 'TTB','TIDLOR','SCB','TLI']
other = ['ADVANC', 'AOT', 'CBG', 'GLOBAL', 'IVL', 'MINT', 'SCC', 'TU', 'WHA','SCGP','ITC']


In [32]:
pe_data = pd.read_csv('pe_ratio_2022.csv')
roe_data = pd.read_csv('roe_2022.csv')

In [33]:
def calculate_z_scores(values):
    mean = np.mean(values)
    std = np.std(values)
    return [(value - mean) / std for value in values]

def find_best_stock_from_local_data(stocks, pe_data, roe_data):
    # Merge PE and ROE data on the 'Ticker' column
    merged_data = pd.merge(pe_data, roe_data, on='TICKER')
    
    # Filter the merged data for the stocks of interest
    filtered_data = merged_data[merged_data['TICKER'].isin(stocks)]
    
    if filtered_data.empty:
        return "No data available for the selected stocks."
    
    pe_ratios = filtered_data['Avg_Annual_PE'].tolist()
    roe_ratios = filtered_data['Avg_Annual_ROE'].tolist()
    
    # Calculate Z-scores for PE and ROE ratios
    pe_z_scores = calculate_z_scores(pe_ratios)
    roe_z_scores = calculate_z_scores(roe_ratios)
    
    # Calculate combined scores and find the stock with the lowest score
    best_stock = None
    best_score = float('inf')
    
    counter = 0  # Initialize a counter for list indexing
    for _, row in filtered_data.iterrows():
        pe_z = pe_z_scores[counter]
        roe_z = roe_z_scores[counter]
        score = (0.5 * pe_z) - (0.5 * roe_z)
        if score < best_score:
            best_stock = row['TICKER']
            best_score = score
        counter += 1  # Increment the counter
    
    return best_stock, best_score

# Example usage
best_tech_stock, best_tech_score = find_best_stock_from_local_data(tech, pe_data, roe_data)
best_energy_stock, best_energy_score = find_best_stock_from_local_data(energy, pe_data, roe_data)
best_healthCare_stock, best_healthCare_score = find_best_stock_from_local_data(healthCare, pe_data, roe_data)
best_PublicUtility_stock, best_PublicUtility_score = find_best_stock_from_local_data(PublicUtility, pe_data, roe_data)
best_Finance_stock, best_Finance_score = find_best_stock_from_local_data(Finance, pe_data, roe_data)
best_other_stock, best_other_score = find_best_stock_from_local_data(other, pe_data, roe_data)

In [34]:
print(f"Best tech stock: {best_tech_stock} with a score of {best_tech_score}")
print(f"Best energy stock: {best_energy_stock} with a score of {best_energy_score}")
print(f"Best healthCare stock: {best_healthCare_stock} with a score of {best_healthCare_score}")
print(f"Best PublicUtility stock: {best_PublicUtility_stock} with a score of {best_PublicUtility_score}")
print(f"Best Finance stock: {best_Finance_stock} with a score of {best_Finance_score}")
print(f"Best other stock: {best_other_stock} with a score of {best_other_score}")

Best tech stock: INTUCH with a score of -1.0
Best energy stock: TOP with a score of -1.2422530700464274
Best healthCare stock: BDMS with a score of -3.3306690738754696e-16
Best PublicUtility stock: HMPRO with a score of -1.3881536691033045
Best Finance stock: TISCO with a score of -0.7484304793826542
Best other stock: IVL with a score of -0.8958508477787475


In [35]:
best_tech_stock = best_tech_stock + '.BK'
best_energy_stock = best_energy_stock + '.BK'
best_healthCare_stock = best_healthCare_stock + '.BK'
best_PublicUtility_stock = best_PublicUtility_stock + '.BK'
best_Finance_stock = best_Finance_stock + '.BK'
best_other_stock = best_other_stock + '.BK'

In [36]:
start = '2022-01-01'
end = '2023-01-01'


tech_df = yf.download(best_tech_stock, start=start,end=end)
energy_df = yf.download(best_energy_stock, start=start,end=end)
healthCare_df = yf.download(best_healthCare_stock, start=start,end=end)
PublicUtility_df = yf.download(best_PublicUtility_stock, start=start,end=end)
Finance_df = yf.download(best_Finance_stock, start=start,end=end)
other_df = yf.download(best_other_stock, start=start,end=end)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [37]:
import numpy as np

# Function to calculate EMA20, EMA200, and generate EMA signals
def calculate_ema_signals(df):
    df['ema20'] = ta.ema(df['Close'], length=5)
    df['ema200'] = ta.ema(df['Close'], length=20)
    df.dropna(inplace=True)  # Remove rows with NaN values to ensure accurate signal calculation
    df['EMA_Signal'] = np.where(df['ema20'] > df['ema200'], 1, 0)
    df['EMA_Signal'] = np.where(df['ema20'] < df['ema200'], -1, df['EMA_Signal'])
    return df

# Dictionary of sector DataFrames
sector_dfs = {
    'tech': tech_df,
    'energy': energy_df,
    'healthCare': healthCare_df,
    'publicUtility': PublicUtility_df,
    'finance': Finance_df,
    'other': other_df
}

# Apply the EMA calculation and signal generation function to each sector DataFrame
for sector, df in sector_dfs.items():
    sector_dfs[sector] = calculate_ema_signals(df)

# Now, each DataFrame in sector_dfs has EMA20, EMA200, and EMA_Signal columns calculated and added.

In [38]:
#RSI
def add_rsi(df, column='Close', length=14):
    df['RSI'] = ta.rsi(df[column], length=length)

dataframes = [tech_df, energy_df, healthCare_df, PublicUtility_df, Finance_df, other_df]

for df in dataframes:
    add_rsi(df)


tech_df.dropna(inplace=True)
energy_df.dropna(inplace=True)
healthCare_df.dropna(inplace=True)
PublicUtility_df.dropna(inplace=True)
Finance_df.dropna(inplace=True)
other_df.dropna(inplace=True)

#if RSI > 70 then the stock is overbought
#if RSI < 30 then the stock is oversold

#if RSI > 0 and RSI < 70 create a new column called 'RSI_Signal' and set it to 1 else set it to 0
def add_rsi_signal(df):
    df['RSI_Signal'] = np.where((df['RSI'] > 30) & (df['RSI'] < 75), 1, 0)

dataframes = [tech_df, energy_df, healthCare_df, PublicUtility_df, Finance_df, other_df]

for df in dataframes:
    add_rsi_signal(df)




In [39]:
def add_macd_signals(df):
    # Calculate MACD using pandas_ta
    a = 12
    b = 26
    c = 9
    EMA_short = df['Close'].ewm(span=a, adjust=False,min_periods= a+1).mean()
    EMA_long = df['Close'].ewm(span=b, adjust=False,min_periods= b+1).mean()

    MACD = EMA_short - EMA_long
    df['macd'] = MACD
    df['signal_line'] = MACD.ewm(span=c, adjust=False,min_periods=c+1).mean()
    
    # Ensure the column names match those from pandas_ta
    df['MACD_Signal'] = np.where(df['macd'] > df['signal_line'], 1,
                                 np.where(df['macd'] < df['signal_line'], -1, 0))
    
    # Drop any rows with NaN values
    df.dropna(inplace=True)
    
    return df




# List of sector DataFrames
sector_dfs = [tech_df, energy_df, healthCare_df, PublicUtility_df, Finance_df, other_df]

# Apply the function to each DataFrame
for df in sector_dfs:
    add_macd_signals(df)


In [40]:
def add_stoch_signals(df):
    stoch = ta.stoch(high=df['High'], low=df['Low'], close=df['Close'], length=14)
    df['%K'] = stoch['STOCHk_14_3_3']
    df['%D'] = stoch['STOCHd_14_3_3']
    df['Stoch_Signal'] = np.where((df['%K'] > df['%D']), 1, 0)
    df['Stoch_Signal'] = np.where((df['%K'] < df['%D']), -1, df['Stoch_Signal'])

    df.dropna(inplace=True)

# List of sector DataFrames
sector_dfs = [tech_df, energy_df, healthCare_df, PublicUtility_df, Finance_df, other_df]

# Apply the function to each DataFrame
for df in sector_dfs:
    add_stoch_signals(df)


In [41]:
import numpy as np

def add_buy_signal(df):
    # Example weights (you might need to determine these based on historical analysis)
    ema_weight = 0.4
    rsi_weight = 0.3
    macd_weight = 0.2
    stoch_weight = 0.1
    
    # Adjust weights based on EMA signal
    ema_weight_adjusted = np.where(df['EMA_Signal'] == -1, 0.5, ema_weight)
    # macd_weight_adjust = np.where(df['MACD_Signal'] == -1 , 0.3 , macd_weight)
    
    # Combining signals with weights
    df['Signal_Score'] = (ema_weight_adjusted * df['EMA_Signal']) + (rsi_weight * df['RSI_Signal']) + (macd_weight * df['MACD_Signal']) + (stoch_weight * df['Stoch_Signal'])
    
    # Generating Buy Signal based on the combined signal score
    df['Buy_Signal'] = np.where(df['Signal_Score'] >= 0.7, 1, np.where(df['Signal_Score'] <= -0.4, -1, 0))
    
    return df

# List of DataFrames
dfs = [tech_df, energy_df, healthCare_df, PublicUtility_df, Finance_df, other_df]

# Apply the function to each DataFrame
for df in dfs:
    df = add_buy_signal(df)


In [42]:
def calculate_forward_returns(df):
    df['Forward Returns'] = np.log(df['Adj Close']).diff().shift(-1)

# List of sector DataFrames
sector_dfs = [tech_df, energy_df, healthCare_df, PublicUtility_df, Finance_df, other_df]

# Apply the function to each DataFrame
for df in sector_dfs:
    calculate_forward_returns(df)

In [43]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Sample DataFrames for illustration (replace these with actual data)
# tech_df = pd.read_csv('path_to_tech_data.csv')
# energy_df = pd.read_csv('path_to_energy_data.csv')
# healthCare_df = pd.read_csv('path_to_healthcare_data.csv')
# PublicUtility_df = pd.read_csv('path_to_publicutility_data.csv')
# Finance_df = pd.read_csv('path_to_finance_data.csv')
# other_df = pd.read_csv('path_to_other_data.csv')

start = 10000

def calculate_max_drawdown(cumulative_returns):
    # Calculate the running maximum
    running_max = np.maximum.accumulate(cumulative_returns)
    # Calculate the drawdown
    drawdown = cumulative_returns / running_max - 1
    # Find the maximum drawdown
    max_drawdown = drawdown.min()
    return max_drawdown

def buy_and_hold_improved(df, start):
    balance = start
    shares = 0
    holding = False  # Flag to track if we are currently holding shares
    cumulative_returns = [start]

    for i in range(len(df)):
        if df['Buy_Signal'].iloc[i] == 1 and not holding:
            shares = balance / df['Close'].iloc[i]
            balance = 0
            holding = True
        elif df['Buy_Signal'].iloc[i] == -1 and holding:
            balance = shares * df['Close'].iloc[i]
            shares = 0
            holding = False
        cumulative_returns.append(balance + shares * df['Close'].iloc[i])

    # If still holding shares at the end, sell them at the last available price
    if holding:
        balance = shares * df['Close'].iloc[-1]

    max_drawdown = calculate_max_drawdown(cumulative_returns)
    return balance, max_drawdown

# Calculate and print results
results = {
    'Tech': buy_and_hold_improved(tech_df, start),
    'Energy': buy_and_hold_improved(energy_df, start),
    'HealthCare': buy_and_hold_improved(healthCare_df, start),
    'PublicUtility': buy_and_hold_improved(PublicUtility_df, start),
    'Finance': buy_and_hold_improved(Finance_df, start),
    'Other': buy_and_hold_improved(other_df, start)
}

for sector, (balance, max_drawdown) in results.items():
    print(f'{sector}: Final Balance: {balance}, Max Drawdown: {max_drawdown:.2%}')

# Calculate total return
def calculate_return():
    return sum(balance for balance, _ in results.values())

initial_investment = start * len(results)
total_profit = calculate_return() - initial_investment
total_return_percentage = (total_profit / initial_investment) * 100

print('\n')
print(f'Total Profit: {total_profit}')
print(f'Total Return: {total_return_percentage:.2f} %')
#print avg max drawdown
max_drawdowns = [max_drawdown for _, max_drawdown in results.values()]
avg_max_drawdown = sum(max_drawdowns) / len(max_drawdowns)
print(f'Average Max Drawdown from all sector: {avg_max_drawdown:.2%}')


Tech: Final Balance: 10086.197426243063, Max Drawdown: -12.53%
Energy: Final Balance: 10008.047810483169, Max Drawdown: -16.24%
HealthCare: Final Balance: 10376.527487805035, Max Drawdown: -8.76%
PublicUtility: Final Balance: 10870.486158769507, Max Drawdown: -4.90%
Finance: Final Balance: 11005.118242612172, Max Drawdown: -2.88%
Other: Final Balance: 9157.838998530942, Max Drawdown: -15.91%


Total Profit: 1504.216124443883
Total Return: 2.51 %
Average Max Drawdown from all sector: -10.20%
